In [2]:
import sys

import blosc
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
sys.path.append('..')

from batch_generator import TwoMnistBatch
from dataset import Dataset
from dataset.opensets import MNIST

In [3]:
dset = MNIST(batch_class=TwoMnistBatch)

ExtractingExtractingExtracting   /tmp/train-images-idx3-ubyte.gz/tmp/train-labels-idx1-ubyte.gz/tmp/t10k-images-idx3-ubyte.gz


Extracting /tmp/t10k-labels-idx1-ubyte.gz


In [4]:
ppl = (dset.train.p
       .normalize_images()
       .concat_and_colorize())

In [5]:
a = ppl.next_batch(15)
plt.imshow(a.images[0])
print(a.labels[0])

TypeError: init_func() got an unexpected keyword argument 'components'